In [35]:
from numpy import transpose
import math
import random
import copy
from pandas import *
from matplotlib import pyplot as plt

def deferred_acceptance(copy_men_values, women_values):
    
    men_values = copy_men_values
    matches = {}
    
    for i in range(len(women_values)):
        matches[i] = []
        
    iter=0
    
    while iter < len(men_values):
        
        men_temp_max_choices = [-1 for i in range(len(men_values))]
        women_temp_max_choices = [-1 for i in range(len(women_values))]
        
        for i in range(len(men_values)):
            temp_max_value = -1
            for j in range(len(men_values[0])):
                if men_values[i][j] > temp_max_value:
                    temp_max_value = men_values[i][j]
                    men_temp_max_choices[i] = j
            matches[men_temp_max_choices[i]].append(i)

        for key, value in matches.items():
            temp_value = -1
            for i in range(len(value)):
                if women_values[key][value[i]] > temp_value:
                    temp_value = women_values[key][value[i]]
                    women_temp_max_choices[key] = value[i]

        stable_matches ={}
        for i in range(len(women_temp_max_choices)):
            stable_matches[i] = [women_temp_max_choices[i]]

        for i in range(len(stable_matches)):
            for j in range(len(matches[i])):
                if matches[i][j] not in stable_matches[i]:
                    men_values[matches[i][j]][i] =0
        iter+=1
        
    return stable_matches

def average_men_value(stable_matches, men_values):
    sum_values = 0
    count=0
    for key in stable_matches:
        if stable_matches[key][0] != -1:
            sum_values+=men_values[stable_matches[key][0]][key]
            count+=1
    return sum_values / count
        
def average_women_value(stable_matches, women_values):
    sum_values = 0
    count=0
    for key in stable_matches:
        if stable_matches[key][0] != -1:
            sum_values+=women_values[key][stable_matches[key][0]]
            count+=1
    return sum_values / count

def propose_diff(men_welfare, women_welfare):
    proposals = [[0 for i in range(len(men_welfare))] for j in range(len(men_welfare[0]))]
    for i in range(len(men_welfare)):
        for j in range(len(men_welfare[0])):
            proposals[i][j] = men_welfare[i][j] - women_welfare[j][i]
    return proposals

def take_away_man(men_welfare):
#     from man's perspective
    differences_men = [[0 for i in range(len(men_welfare))] for j in range(len(men_welfare[0]))]
    for i in range(len(men_welfare) - 1):
        for j in range(len(men_welfare)):
            differences_men[i][j] = men_welfare[i][j] - men_welfare[i+1][j]
    return differences_men

def take_away_woman(women_welfare):
#     from woman's perspective
    differences_women = [[0 for i in range(len(women_welfare))] for j in range(len(women_welfare[0]))]
    for i in range(len(women_welfare)):
        for j in range(len(women_welfare)-1):
            differences_women[i][j] = women_welfare[i][j] - women_welfare[i][j+1]
    return differences_women
    
def main():
    
    number_men = 5
    number_women = 5
    men_values = [[random.uniform(0,1) for i in range(number_women)] for j in range(number_men)]
    women_values = [[random.uniform(0,1) for i in range(number_men)] for j in range(number_women)]
    
    for i in range(5):
        men_values[0][i]=1
    print(DataFrame(men_values))
    print(DataFrame(women_values))

    copy_men_values = copy.deepcopy(men_values)
    original_men_values = copy.deepcopy(men_values)
    stable_matches = deferred_acceptance(copy_men_values, women_values)
    
    array_values = [0]*number_men
    count=0
    for key in stable_matches:
        if stable_matches[key][0] != -1:
            array_values[stable_matches[key][0]]=men_values[stable_matches[key][0]][key]
    
    del men_values[0]
    copy_men_values = copy.deepcopy(men_values)
    new_stable_matches = deferred_acceptance(copy_men_values, women_values)
    
    new_array_values = [0]*(number_men -1)
    count=0
    for key in stable_matches:
        if new_stable_matches[key][0] != -1:
            new_array_values[new_stable_matches[key][0]]=men_values[new_stable_matches[key][0]][key]
        
    nonproposing_men_values = transpose(original_men_values)
    proposing_women_values = transpose(women_values)
    new_new_stable_matches = deferred_acceptance(proposing_women_values, nonproposing_men_values)
    
    new_new_array_values = [0]*number_men
    count=0
    for key in new_new_stable_matches:
        if key!= -1 and new_new_stable_matches[key][0] != -1:
            new_new_array_values[key]=nonproposing_men_values[new_new_stable_matches[key][0]][key]
            
    print(array_values) # n, proposing
    print(new_array_values) # n-1, proposing
    print(new_new_array_values) # n, not proposing
    
    better = 0
    same = 0
    worse = 0

    for i in range(len(new_array_values)):
        if new_array_values[i] > array_values[i+1] :
            better +=1
        elif new_array_values[i] == array_values[i+1]:
            same+=1
        else:
            worse +=1
    
    print(f"Dropping man: \n Better: {better} Worse: {worse} Same: {same}")
    
    
    

    
main()    

          0         1         2         3         4
0  1.000000  1.000000  1.000000  1.000000  1.000000
1  0.381627  0.969966  0.321918  0.423558  0.250159
2  0.286357  0.987765  0.828849  0.810092  0.844370
3  0.006896  0.004401  0.503000  0.180499  0.396899
4  0.362868  0.144351  0.255644  0.818721  0.056927
          0         1         2         3         4
0  0.387179  0.778402  0.665218  0.287644  0.870468
1  0.697649  0.658914  0.304501  0.908051  0.137735
2  0.126307  0.427287  0.158105  0.637478  0.206859
3  0.756460  0.159464  0.895195  0.943836  0.089437
4  0.972888  0.622024  0.692326  0.419244  0.414593
[1, 0.9699663944785482, 0.8443698018776769, 0.5030000130152932, 0.8187212689248552]
[0.9699663944785482, 0.8443698018776769, 0.5030000130152932, 0.8187212689248552]
[1.0, 0.4235579052045624, 0.8443698018776769, 0.5030000130152932, 0.36286812308438987]
